<a href="https://colab.research.google.com/github/jasonstoy/TensorFlow/blob/main/11.%20Transfer%20Learning%20(feature_extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [ ]:
! nvidia-smi

In [ ]:
import zipfile

In [ ]:
file = zipfile.ZipFile("10_food_classes_10_percent.zip")
file.extractall()
file.close()

In [ ]:
import os

for dir_path, dir_name, file_name in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(file_name)} files and {len(dir_name)} folders in {dir_path}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setup data loader
BATCH_SIZE = 32
IMAGE_SHAPE = (224,224)

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=0.2, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=5)

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             class_mode="categorical",
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             seed=5)

In [ ]:
# setting up callbacks for tensorboard
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir) 
  return tensorboard_callback


In [ ]:
# creat model from tensorflow hub
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
def create_model(model_url, num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extraction_layer",
                                           input_shape=IMAGE_SHAPE+(3,))
  
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, 
                 activation="softmax",
                 name="output_layer")
  ])

  return model

In [ ]:
# create resnet model
resnet_model = create_model(resnet_url, 10)

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
history_resnet = resnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data = test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                         experiment_name="resnet50v2")])

In [ ]:
import pandas as pd
pd.DataFrame({"train_loss": history_resnet.history['loss'],
              "val_loss": history_resnet.history['val_loss']}).plot()

In [ ]:
pd.DataFrame({"train_accuracy": history_resnet.history['accuracy'],
              "val_accuracy": history_resnet.history['val_accuracy']}).plot()

In [ ]:
# create efficient model
efficient_model = create_model(efficientnet_url, 10)

In [ ]:
efficient_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
history_efficient_model = efficient_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data = test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                         experiment_name="efficient_model")])

In [ ]:
pd.DataFrame({"train_loss": history_resnet.history['loss'],
              "val_loss": history_resnet.history['val_loss']}).plot()

In [ ]:
pd.DataFrame({"train_accuracy": history_resnet.history['accuracy'],
              "val_accuracy": history_resnet.history['val_accuracy']}).plot()

In [ ]:
# compare results in tensorboard
! tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EfficientNetB0 vs ResNet50V2" \
  --description "Compare two feature extraction architecture" \
  --one_shot

In [ ]:
! tensorboard dev list